# Введение в обработку естественного языка
## Урок 7. Сверточные нейронные сети для анализа текста
Берем отзывы за лето (из архива с материалами или предыдущего занятия)

Учим conv сеть для классификации
Рассмотреть 2-а варианта сеточек:
Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)
Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [1]:
!pip install pymorphy2

  Using cached pymorphy2-0.9.1-py3-none-any.whl (55 kB)
  Using cached docopt-0.6.2.tar.gz (25 kB)
  Preparing metadata (setup.py) ... done
  Using cached pymorphy2_dicts_ru-2.4.417127.4579844-py2.py3-none-any.whl (8.2 MB)
  Using cached DAWG_Python-0.7.2-py2.py3-none-any.whl (11 kB)
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=0710343ba7c54af4488fc0c84e77f8515c73074493e40e342ba7f322f0ace7ee
  Stored in directory: /home/alexey/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
!pip install stop-words

  Using cached stop-words-2018.7.23.tar.gz (31 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32895 sha256=8c716881b7d76e0cbb1ce992f1d7ccb5431af36cdbeb771c12e358a1d746a817
  Stored in directory: /home/alexey/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop-words


In [6]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

In [8]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
#from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.metrics import categorical_crossentropy
from keras.callbacks import EarlyStopping

import tensorflow as tf

In [9]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/alexey/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
df = pd.read_excel('summer_reports.xls')

In [15]:
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [16]:
df.drop('Date', axis=1, inplace=True)

In [17]:
df_train, for_test = train_test_split(df, test_size=0.35,  shuffle=True)
df_test, df_val = train_test_split(for_test, test_size=0.42, shuffle=True)

In [18]:
df_train.head(5)

,Rating,Content
12601,5,В целом удобно
2584,5,Отличное приложение! Удобно и быстро!
10166,5,Пока работает на отлично.
14478,5,Ок
13800,5,Не работает вход по сканеру пальца на xiaomi 4...


In [19]:
max_words = 2000 
max_len = 20
num_classes = 6

In [20]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

In [21]:
def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [22]:
df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_val['Content'] = df_val['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [23]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()

In [24]:
tokens = word_tokenize(train_corpus)

In [25]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [26]:
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [27]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [28]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [29]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["Content"]], dtype=np.int32)

In [30]:
x_train.shape, max_len

((13428, 20), 20)

In [31]:
y_train = tf.keras.utils.to_categorical(df_train["Rating"], num_classes)
y_val = tf.keras.utils.to_categorical(df_val["Rating"], num_classes)

In [32]:
epochs = 20
batch_size = 16
print_batch_n = 100

In [33]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 5))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(num_classes))
model.add(Activation('softmax'))


model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

2023-05-12 14:39:49.687340: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-12 14:39:49.688764: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [34]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss') 

In [35]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
756/756 [==============================] - 7s 8ms/step - loss: 0.7175 - accuracy: 0.7593 - val_loss: 0.6497 - val_accuracy: 0.7937
Epoch 2/20
756/756 [==============================] - 6s 8ms/step - loss: 0.5744 - accuracy: 0.7999 - val_loss: 0.6544 - val_accuracy: 0.7826


In [36]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

190/190 [==============================] - 1s 4ms/step - loss: 0.6701 - accuracy: 0.7614


Test score: 0.6700571775436401
Test accuracy: 0.7613561749458313


In [37]:
def different_convolutions(conv=(2,2)):
    results = []
    for temp_conv in range(conv[0], conv[1]+1):
        model = Sequential()
        model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
        model.add(Conv1D(128, temp_conv))
        model.add(Activation("relu"))
        model.add(GlobalMaxPool1D())
        model.add(Activation("relu"))
        model.add(Dense(num_classes))
        model.add(Activation('softmax'))

        model.compile(
            loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
        )
        tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
        early_stopping=EarlyStopping(monitor='val_loss')  


        history = model.fit(x_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_split=0.1,
                            callbacks=[tensorboard, early_stopping])
        # results.append((history.history, temp_conv))
        results.append(history.history)
    return results, [i for i in range(conv[0], conv[1]+1)]

In [38]:
res = different_convolutions((1,15))

Epoch 1/20
756/756 [==============================] - 45s 59ms/step - loss: 0.7370 - accuracy: 0.7576 - val_loss: 0.6527 - val_accuracy: 0.7863
Epoch 2/20
756/756 [==============================] - 45s 60ms/step - loss: 0.5893 - accuracy: 0.7922 - val_loss: 0.6467 - val_accuracy: 0.7915
Epoch 1/20
756/756 [==============================] - 7s 9ms/step - loss: 0.7283 - accuracy: 0.7542 - val_loss: 0.6445 - val_accuracy: 0.7893
Epoch 2/20
756/756 [==============================] - 8s 11ms/step - loss: 0.5863 - accuracy: 0.7972 - val_loss: 0.6515 - val_accuracy: 0.7856
Epoch 1/20
756/756 [==============================] - 7s 8ms/step - loss: 0.7260 - accuracy: 0.7584 - val_loss: 0.6327 - val_accuracy: 0.7990
Epoch 2/20
756/756 [==============================] - 8s 11ms/step - loss: 0.5781 - accuracy: 0.8009 - val_loss: 0.6467 - val_accuracy: 0.7893
Epoch 1/20
756/756 [==============================] - 11s 14ms/step - loss: 0.7243 - accuracy: 0.7566 - val_loss: 0.6499 - val_accuracy: 0.787

In [33]:
pd.DataFrame(res[0], index=res[1])

,val_loss,val_accuracy,loss,accuracy
1,"[0.6451280256393501, 0.6563991670810901]","[0.7714073061943054, 0.7788532972335815]","[0.7484327158653692, 0.598894610008426]","[0.7536616, 0.7910633]"
2,"[0.6329048269734131, 0.6402106578014541]","[0.7795979380607605, 0.7870439291000366]","[0.7397208883538442, 0.5929695876363786]","[0.75233763, 0.7904841]"
3,"[0.6329639810550062, 0.6391972172020624]","[0.7758749127388, 0.7810871005058289]","[0.7313166497584261, 0.5845596031063491]","[0.75556475, 0.79528344]"
4,"[0.6476948708184868, 0.6484966797342251]","[0.7736411094665527, 0.7751303315162659]","[0.7296610081965241, 0.5829357847676181]","[0.75482005, 0.7976003]"
5,"[0.6513570446357386, 0.6549553111339805]","[0.7706627249717712, 0.7743856906890869]","[0.7329112542923394, 0.5831952583656659]","[0.754158, 0.79247]"
6,"[0.6642446905840536, 0.652454979590134, 0.7101...","[0.767684280872345, 0.7773641347885132, 0.7691...","[0.7315550524413117, 0.5866512411430842, 0.474...","[0.75763345, 0.7963591, 0.84377325]"
7,"[0.6377961953806505, 0.6639943398474226]","[0.7751303315162659, 0.7736411094665527]","[0.7291232948072399, 0.5792362760624513]","[0.7556475, 0.7974348]"
8,"[0.6499771976417689, 0.6550665344219109]","[0.7781087160110474, 0.782576322555542]","[0.7336939016846423, 0.587226091634157]","[0.75233763, 0.79652464]"
9,"[0.6387028434519793, 0.6689053216445651]","[0.7721518874168396, 0.7661950588226318]","[0.7359129328346963, 0.5870467721108913]","[0.7529996, 0.799669]"
10,"[0.6583572827606883, 0.6744436650070252]","[0.7751303315162659, 0.7773641347885132]","[0.7339794991082478, 0.5855140246879379]","[0.7513446, 0.7947042]"


In [42]:
res

([{'loss': [0.7369810342788696, 0.5892660617828369],
   'accuracy': [0.7575506567955017, 0.7922217845916748],
   'val_loss': [0.6527249217033386, 0.6466626524925232],
   'val_accuracy': [0.7862993478775024, 0.7915115356445312]},
  {'loss': [0.72834312915802, 0.5862574577331543],
   'accuracy': [0.7542408108711243, 0.7971866130828857],
   'val_loss': [0.6445209383964539, 0.651465117931366],
   'val_accuracy': [0.7892777323722839, 0.7855547070503235]},
  {'loss': [0.7259688973426819, 0.5781187415122986],
   'accuracy': [0.7583781480789185, 0.800910234451294],
   'val_loss': [0.632665753364563, 0.6466896533966064],
   'val_accuracy': [0.7989575862884521, 0.7892777323722839]},
  {'loss': [0.7242674231529236, 0.5758451223373413],
   'accuracy': [0.7565577030181885, 0.8026478886604309],
   'val_loss': [0.6499274373054504, 0.646302342414856],
   'val_accuracy': [0.7877885103225708, 0.7855547070503235]},
  {'loss': [0.7228543162345886, 0.5744708180427551],
   'accuracy': [0.756805956363678, 0.

In [49]:
import numpy as np
weights = model.layers[0].get_weights()[0]
np.savetxt('model.txt', weights ,fmt='%s')

In [50]:
with open('model.txt', 'r', encoding='utf8') as f:
    pretrained_words = f.readlines()
pretrained_words.pop(0)

'-0.015095357 -0.03959214 0.008185286 0.0007301944 -0.026648914 0.045232702 0.057778858 -0.034507044 -0.04752925 -0.017102094 -0.03354258 -0.0012690077 0.0053098802 0.011524156 0.009314574 -0.046989504 -0.024000794 0.0037724348 -0.0326308 0.003384432 0.021586683 0.023431601 -0.046708558 0.046676572 -0.030239554 0.05787146 -0.02712383 -0.011638471 0.0191966 -0.018725017 -0.028058896 -0.053002395 -0.02642748 -0.057575084 0.026125183 0.025924707 -0.04164207 0.024537656 0.020818828 0.044692088 0.015409228 0.040675517 -0.05298462 0.018757353 0.015137646 0.033983435 -0.020970993 -0.00360053 0.03602773 0.010452118 -0.023494061 0.011337303 -0.041881897 -0.026350146 -0.04010025 0.0046434486 0.031548936 -0.0594734 0.044558726 -0.009634795 -0.005723461 0.03493536 -0.027057214 0.040231448 0.012951486 0.010381254 0.05406473 0.0060528503 -0.024338583 0.015542297 0.06597497 0.009625832 -0.04795719 0.033162236 0.02321363 0.0059979316 -0.014784107 0.03840651 0.0152088795 0.017882956 0.023688763 0.00561

In [51]:
pretrained_words[1].split()[0].split('_')[0]

'-0.06233946'

In [52]:
np.asarray(pretrained_words[1].split()[1:], dtype='float32').shape

(127,)

In [53]:
counter = 0
embeddings_index = {}
for pre_word in pretrained_words:
    counter += 1
    word = pre_word.split()[0].split('_')[0]
    coefs = np.asarray(pre_word.split()[1:], dtype='float32')
    embeddings_index[word] = coefs
print(counter)

1999


In [54]:
EMBEDDING_DIM = 300

In [55]:
embedding_matrix = np.zeros((len(vocabulary) + 1, EMBEDDING_DIM))
for word, i in vocabulary.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [63]:
embedding_matrix.shape

(2000, 300)

In [78]:
embedding_layer = Embedding(input_dim = len(vocabulary) + 1, 
                            output_dim = EMBEDDING_DIM,
                            weights = [embedding_matrix],
                            input_length=max_len,
                            trainable = False)

In [79]:
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(128, 5))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(50))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_24 (Embedding)    (None, 20, 300)           600000    
                                                                 
 conv1d_16 (Conv1D)          (None, 16, 128)           192128    
                                                                 
 activation_47 (Activation)  (None, 16, 128)           0         
                                                                 
 global_max_pooling1d_16 (Gl  (None, 128)              0         
 obalMaxPooling1D)                                               
                                                                 
 dense_16 (Dense)            (None, 50)                6450      
                                                                 
 activation_48 (Activation)  (None, 50)                0         
                                                     

In [80]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [81]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
756/756 [==============================] - 6s 8ms/step - loss: 1.4849 - accuracy: 0.7046 - val_loss: 1.2391 - val_accuracy: 0.7185
Epoch 2/20
756/756 [==============================] - 7s 9ms/step - loss: 1.1436 - accuracy: 0.7053 - val_loss: 1.0489 - val_accuracy: 0.7185


Вывод: Результат модели с предобученным эмбеддингом оказался чуть ниже